In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from keras.applications.resnet50 import ResNet50

import os
from os import walk
import glob as gb
import numpy as np
import seaborn as sns
import cv2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(50000, 32, 32, 3) (50000, 1)
(10000, 32, 32, 3) (10000, 1)


In [3]:
X_train,X_val,Y_train,Y_val=train_test_split(X_train,Y_train,test_size=.2)
print(X_train.shape, Y_train.shape)
print(X_val.shape, Y_val.shape)

(40000, 32, 32, 3) (40000, 1)
(10000, 32, 32, 3) (10000, 1)


In [4]:
X_train = X_train.astype('float32') / 255
X_val = X_val.astype('float32') / 255
X_test = X_test.astype('float32') / 255

print(X_train.shape, Y_train.shape)
print(X_val.shape, Y_val.shape)
print(X_test.shape, Y_test.shape)

SEED = 1000
IMG_SIZE = 50
BATCH_SIZE = 128

(40000, 32, 32, 3) (40000, 1)
(10000, 32, 32, 3) (10000, 1)
(10000, 32, 32, 3) (10000, 1)


In [5]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [6]:
def identity_block(X, filters):
    f1, f2, f3 = filters
    X_copy = X
    
    # 1st Layer
    X = layers.Conv2D(filters=f1, kernel_size=(1,1), strides=(1,1), padding='valid')(X)
    X = layers.BatchNormalization(axis=3)(X)
    X = layers.Activation('relu')(X)
    
    # 2nd Layer
    X = layers.Conv2D(filters=f2, kernel_size=(3,3), strides=(1,1), padding='same')(X)
    X = layers.BatchNormalization(axis=3)(X)
    X = layers.Activation('relu')(X) 
    
    # 3rd Layer
    X = layers.Conv2D(filters=f3, kernel_size=(1,1), strides=(1,1), padding='valid')(X)
    X = layers.BatchNormalization(axis=3)(X)
    
    # Add the Skip COnnection
    X = layers.Add()([X, X_copy])
    X = layers.Activation('relu')(X)
    
    return X

In [7]:
def conv_blocks(X, filters, s=2):
    f1, f2, f3 = filters
    X_copy = X
    
    # 1st Layer
    X = layers.Conv2D(filters=f1, kernel_size=(1,1), strides=(s,s), padding='valid')(X)
    X = layers.BatchNormalization(axis=3)(X)
    X = layers.Activation('relu')(X)
    
    # 2nd Layer
    X = layers.Conv2D(filters=f2, kernel_size=(3,3), strides=(1,1), padding='same')(X)
    X = layers.BatchNormalization(axis=3)(X)
    X = layers.Activation('relu')(X) 
    
    # 3rd Layer
    X = layers.Conv2D(filters=f3, kernel_size=(1,1), strides=(1,1), padding='valid')(X)
    X = layers.BatchNormalization(axis=3)(X)
    
    ### match the dimension
    X_copy = layers.Conv2D(filters=f3, kernel_size=(1,1), strides=(s,s), padding='valid')(X_copy)
    X_copy = layers.BatchNormalization(axis=3)(X_copy)
    
    # Add the Skip COnnection
    X = layers.Add()([X, X_copy])
    X = layers.Activation('relu')(X)
    
    return X

In [8]:
def ResNet50():
    X_input = keras.Input((32, 32, 3))
    X = layers.ZeroPadding2D((3,3))(X_input)
    
    # Satge Conv1
    X = layers.Conv2D(64, (7,7), strides=(2,2))(X)
    X = layers.BatchNormalization(axis=3)(X)
    X = layers.Activation('relu')(X) 
    X = layers.MaxPooling2D((3,3), strides=(2,2))(X)
    
    # stage Conv2_x
    X = conv_blocks(X, filters=[64,64,256], s=1)
    X = identity_block(X, filters=[64,64,256])
    X = identity_block(X, filters=[64,64,256])
    
    # stage Conv3_x
    X = conv_blocks(X, filters=[128,128,512], s=2)
    X = identity_block(X, filters=[128,128,512])
    X = identity_block(X, filters=[128,128,512])
    X = identity_block(X, filters=[128,128,512])
    
    # stage Conv4_x
    X = conv_blocks(X, filters=[256,256,1024], s=2)
    X = identity_block(X, filters=[256,256,1024])
    X = identity_block(X, filters=[256,256,1024])
    X = identity_block(X, filters=[256,256,1024])
    X = identity_block(X, filters=[256,256,1024])
    X = identity_block(X, filters=[256,256,1024])
    
    # stage Conv5_x
    X = conv_blocks(X, filters=[512,512,2048], s=2)
    X = identity_block(X, filters=[512,512,2048])
    X = identity_block(X, filters=[512,512,2048])
    
    
    # X = layers.AveragePooling2D((2,2))(X)
    X = layers.Flatten()(X)
    X = layers.Dense(10, activation='softmax', kernel_initializer='he_normal')(X)
    
    model = keras.Model(inputs=X_input, outputs=X, name='ResNet50')
    
    return model

In [9]:
model = ResNet50()
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 16, 16, 64)   9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 16, 16, 64)   256         conv2d[0][0]                     
___________________________________________________________________________________________

In [10]:
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 40:
        lr *= 0.5e-3
    elif epoch > 30:
        lr *= 1e-3
    elif epoch > 20:
        lr *= 1e-2
    elif epoch > 10:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr
lr_scheduler = LearningRateScheduler(lr_schedule)

In [11]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule(0)), 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

Learning rate:  0.001


In [12]:
h = model.fit(X_train,Y_train, 
              epochs=10,
              steps_per_epoch = 100,
              validation_data = (X_val,Y_val),
              validation_steps=10,
              batch_size=256,
              callbacks=lr_scheduler
             )

Epoch 1/10
Learning rate:  0.001
100/100 [==============================] - 21s 107ms/step - loss: 2.1395 - accuracy: 0.3179 - val_loss: 2.4413 - val_accuracy: 0.1059
Epoch 2/10
Learning rate:  0.001
100/100 [==============================] - 11s 110ms/step - loss: 1.5302 - accuracy: 0.4596 - val_loss: 2.7416 - val_accuracy: 0.1102
Epoch 3/10
Learning rate:  0.001
100/100 [==============================] - 10s 99ms/step - loss: 1.4169 - accuracy: 0.4977 - val_loss: 3.0026 - val_accuracy: 0.1145
Epoch 4/10
Learning rate:  0.001
100/100 [==============================] - 10s 99ms/step - loss: 1.2757 - accuracy: 0.5547 - val_loss: 1.8008 - val_accuracy: 0.3531
Epoch 5/10
Learning rate:  0.001
100/100 [==============================] - 10s 100ms/step - loss: 1.2526 - accuracy: 0.5725 - val_loss: 1.8079 - val_accuracy: 0.3566
Epoch 6/10
Learning rate:  0.001
100/100 [==============================] - 10s 101ms/step - loss: 1.3090 - accuracy: 0.5652 - val_loss: 9.6753 - val_accuracy: 0.2121


#**WIth Augmented Data**

In [13]:
datagen = ImageDataGenerator(
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

In [18]:
train_batch = datagen.flow(
    X_train,
    y=Y_train,
    batch_size=BATCH_SIZE,
    seed=SEED
)

valid_batch = datagen.flow(
    X_val,
    y=Y_val,
    batch_size=BATCH_SIZE,
    seed=SEED
)

In [17]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule(0)), 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

Learning rate:  0.001


In [19]:
h = model.fit(train_batch, 
              epochs=10,
              steps_per_epoch = 100,
              validation_data = valid_batch,
              validation_steps=10,
              batch_size=256,
              callbacks=lr_scheduler
             )

Epoch 1/10
Learning rate:  0.001
100/100 [==============================] - 18s 120ms/step - loss: 2.1178 - accuracy: 0.1010 - val_loss: 645.3227 - val_accuracy: 0.0414
Epoch 2/10
Learning rate:  0.001
100/100 [==============================] - 10s 104ms/step - loss: 2.0151 - accuracy: 0.1048 - val_loss: 3.5037 - val_accuracy: 0.1367
Epoch 3/10
Learning rate:  0.001
100/100 [==============================] - 11s 104ms/step - loss: 2.1974 - accuracy: 0.1038 - val_loss: 310.0392 - val_accuracy: 0.0000e+00
Epoch 4/10
Learning rate:  0.001
100/100 [==============================] - 11s 106ms/step - loss: 2.0168 - accuracy: 0.0998 - val_loss: 377.0410 - val_accuracy: 0.0078
Epoch 5/10
Learning rate:  0.001
100/100 [==============================] - 11s 111ms/step - loss: 1.9798 - accuracy: 0.0934 - val_loss: 9.4737 - val_accuracy: 0.1391
Epoch 6/10
Learning rate:  0.001
100/100 [==============================] - 11s 111ms/step - loss: 1.9781 - accuracy: 0.1166 - val_loss: 2.9053 - val_accur